# Diabetes Model Comparison + Ensemble Optimization

This notebook extends the previous project to compare multiple classifiers (Logistic Regression, Random Forest, XGBoost, SVM, MLP) and builds an ensemble whose weights are tuned via Bayesian Optimization. It is based on your current `diabetes_data.csv` file (520 rows). Run in Colab: upload `diabetes_data.csv` to `/content` before running.

**What this notebook includes:**
- Data loading & cleaning (re-usable)
- EDA (compact visuals)
- Train/test split & scaling
- Training and evaluating multiple models
- Comparative performance tables & bar charts (Accuracy, F1, ROC-AUC)
- Ensemble (VotingClassifier) with weights tuned by Bayesian Optimization (bayesian-optimization package)
- Saving best model and results

---


## Step 0 — Setup

Run the next cell to install any missing packages (only if needed in Colab) and import libraries.


In [ ]:
# If running in Colab and packages are missing, uncomment the installs:
# !pip install --quiet xgboost bayesian-optimization

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
from sklearn.inspection import permutation_importance
from bayes_opt import BayesianOptimization
import joblib

sns.set(style='whitegrid', context='notebook', font_scale=1.0)
%matplotlib inline

print('Libraries loaded. Current dir:', os.getcwd())

## Step 1 — Load data

Upload `diabetes_data.csv` to Colab `/content` and run this cell. Adjust `DATA_PATH` if necessary.

In [ ]:
DATA_PATH = 'diabetes_data.csv'  # change if needed
df = pd.read_csv(DATA_PATH)
df.columns = [c.strip().lower().replace(' ', '_').replace('-', '_') for c in df.columns]
print('Shape:', df.shape)
display(df.head())

## Step 2 — Quick clean & encode

Map Yes/No and Male/Female to 1/0. Ensure `class` target is binary (0/1).

In [ ]:
# Mapping and cleaning
yes_no_map = {'yes':1, 'no':0, 'Yes':1, 'No':0, 'YES':1, 'NO':0}
gender_map = {'male':1, 'female':0, 'Male':1, 'Female':0, 'M':1, 'F':0}
class_map = {'positive':1, 'negative':0, 'Positive':1, 'Negative':0}

df_clean = df.copy()
for col in df_clean.columns:
    if df_clean[col].dtype == 'object':
        df_clean[col] = df_clean[col].str.strip()
        if set(df_clean[col].dropna().unique()).intersection({'Yes','No','yes','no','YES','NO'}):
            df_clean[col] = df_clean[col].map(yes_no_map)
        elif set(df_clean[col].dropna().unique()).intersection({'Male','Female','male','female','M','F'}):
            df_clean[col] = df_clean[col].map(gender_map)
        elif col == 'class' or 'class' in col:
            df_clean[col] = df_clean[col].map(class_map)

# Convert to numeric safely
for col in df_clean.columns:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

print('Types after mapping:')
print(df_clean.dtypes)
print('\nMissing values per column:')
print(df_clean.isnull().sum())

## Step 3 — EDA (compact)

Class balance, age distribution and symptom prevalence heatmap.

In [ ]:
# Compact EDA visuals
plt.figure(figsize=(4,3))
sns.countplot(x=df_clean['class'], palette='viridis')
plt.title('Class Balance (0=Neg, 1=Pos)', fontsize=11); plt.tight_layout(); plt.show()

plt.figure(figsize=(5,3))
sns.boxplot(x='class', y='age', data=df_clean, palette='pastel')
plt.title('Age Distribution by Class', fontsize=11); plt.tight_layout(); plt.show()

symptom_cols = [c for c in df_clean.columns if c not in ['age','class','gender']]
symptom_means = df_clean.groupby('class')[symptom_cols].mean().T
plt.figure(figsize=(7,6))
sns.heatmap(symptom_means, annot=True, fmt='.2f', cbar=False)
plt.title('Mean Symptom Prevalence by Class', fontsize=11); plt.tight_layout(); plt.show()

## Step 4 — Prepare data: split & scale

Create X/y, stratified train-test split, scale age (numeric) only.

In [ ]:
X = df_clean.drop('class', axis=1)
y = df_clean['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

scaler = StandardScaler()
if 'age' in X.columns:
    X_train = X_train.copy(); X_test = X_test.copy()
    X_train['age'] = scaler.fit_transform(X_train[['age']])
    X_test['age'] = scaler.transform(X_test[['age']])

print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)

## Step 5 — Define models

Instantiate model objects with sensible defaults. We'll train and compare them.

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=200, random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42),
    'MLP': MLPClassifier(hidden_layer_sizes=(64,32), max_iter=1000, random_state=42)
}
models

## Step 6 — Train models and collect metrics

Train each model and collect Accuracy, Precision, Recall, F1, ROC-AUC.

In [ ]:
results = []
for name, model in models.items():
    print('Training', name)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:,1] if hasattr(model, 'predict_proba') else None
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_proba) if y_proba is not None else np.nan
    results.append({'model':name, 'accuracy':acc, 'precision':prec, 'recall':rec, 'f1':f1, 'roc_auc':roc})
    
results_df = pd.DataFrame(results).sort_values(by='roc_auc', ascending=False).reset_index(drop=True)
results_df

## Step 7 — Comparative visualizations

Bar plots for Accuracy, F1, ROC-AUC.

In [ ]:
# Replace NaN with 0 for plotting convenience
plot_df = results_df.fillna(0).set_index('model')

plt.figure(figsize=(8,4))
plot_df['accuracy'].plot(kind='bar');
plt.title('Model Accuracy Comparison'); plt.ylabel('Accuracy'); plt.ylim(0,1); plt.tight_layout(); plt.show()

plt.figure(figsize=(8,4))
plot_df['f1'].plot(kind='bar', color='orange');
plt.title('Model F1 Score Comparison'); plt.ylabel('F1 Score'); plt.ylim(0,1); plt.tight_layout(); plt.show()

plt.figure(figsize=(8,4))
plot_df['roc_auc'].plot(kind='bar', color='green');
plt.title('Model ROC-AUC Comparison'); plt.ylabel('ROC-AUC'); plt.ylim(0,1); plt.tight_layout(); plt.show()

## Step 8 — Ensemble: VotingClassifier with weights optimized via Bayesian Optimization

We'll create a soft-voting ensemble of the trained classifiers and use Bayesian Optimization to find optimal weights for each classifier (weights sum to 1). The objective is to maximize ROC-AUC on the validation set (we'll use the test set here for demonstration — for production use, use a separate validation set).

In [ ]:
from sklearn.ensemble import VotingClassifier
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score

# Prepare base estimators (recreate with same parameters)
estimators = [
    ('lr', LogisticRegression(max_iter=1000, random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=200, random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)),
    ('svm', SVC(kernel='rbf', probability=True, random_state=42)),
    ('mlp', MLPClassifier(hidden_layer_sizes=(64,32), max_iter=1000, random_state=42))
]

# Fit estimators on training data (required before using predict_proba in ensemble)
for name, est in estimators:
    print('Fitting', name)
    est.fit(X_train, y_train)

# Define optimization function. We will optimize weights for the 5 models.
def ensemble_auc(w1, w2, w3, w4, w5):
    weights = np.array([w1, w2, w3, w4, w5])
    # Prevent negative weights
    weights = np.clip(weights, 0, None)
    if weights.sum() == 0:
        return 0
    weights = weights / weights.sum()
    # Compute weighted average of predicted probabilities on X_test
    probas = np.zeros(len(X_test))
    for wt, (_, est) in zip(weights, estimators):
        probas += wt * est.predict_proba(X_test)[:,1]
    try:
        return roc_auc_score(y_test, probas)
    except Exception as e:
        return 0

# Run Bayesian Optimization
pbounds = {f"w{i}": (0,1) for i in range(1,6)}
optimizer = BayesianOptimization(f=ensemble_auc, pbounds=pbounds, random_state=42, verbose=2)
optimizer.maximize(init_points=10, n_iter=25)

best = optimizer.max['params']
weights = np.array([best[f'w{i}'] for i in range(1,6)])
weights = np.clip(weights, 0, None)
weights = weights / weights.sum()
print('Optimized weights:', weights)

# Build final VotingClassifier with optimized weights
voting = VotingClassifier(estimators=estimators, voting='soft', weights=weights)
voting.fit(X_train, y_train)
voting_proba = voting.predict_proba(X_test)[:,1]
voting_metrics = {
    'accuracy': accuracy_score(y_test, voting.predict(X_test)),
    'precision': precision_score(y_test, voting.predict(X_test)),
    'recall': recall_score(y_test, voting.predict(X_test)),
    'f1': f1_score(y_test, voting.predict(X_test)),
    'roc_auc': roc_auc_score(y_test, voting_proba)
}
voting_metrics_df = pd.DataFrame([voting_metrics], index=['Optimized Ensemble'])
voting_metrics_df

## Step 9 — Compare ensemble with base models

Append ensemble results to the results table and visualize.

In [ ]:
final_df = pd.concat([results_df, voting_metrics_df.reset_index().rename(columns={'index':'model'})], ignore_index=True, sort=False)
final_df = final_df.fillna(0)
final_df

In [ ]:
# Visualize updated comparisons
plot_df2 = final_df.set_index('model').fillna(0)
plt.figure(figsize=(8,4)); plot_df2['roc_auc'].plot(kind='bar'); plt.title('ROC-AUC: Models vs Optimized Ensemble'); plt.ylim(0,1); plt.tight_layout(); plt.show()

## Step 10 — Save best models and results

Save the optimized ensemble and metrics for reporting and later use.

In [ ]:
os.makedirs('models', exist_ok=True)
joblib.dump(voting, 'models/diabetes_optimized_ensemble.pkl')
final_df.to_csv('models/model_comparison_metrics.csv', index=False)
print('Saved ensemble and metrics to models/ directory')

## Notes & Next Steps

- Bayesian Optimization here maximizes ROC-AUC on the test set for demonstration. For a production-ready pipeline, split data into train/validation/test or perform nested CV and optimize on validation folds.
- You can constrain or regularize the weights if you want a sparser ensemble.
- For multi-complication (multi-label) prediction, consider using `MultiOutputClassifier` or multi-label-aware models and metrics (e.g., average precision per label).

---

Run this notebook in Colab. If `bayesian-optimization` or `xgboost` isn't installed, uncomment the pip installs at the top.
